In [ ]:
from typing import Any, Callable
import threading
import queue

Handler = Callable[[Any], Any]


class WorkPool:
    def __init__(
        self,
        handler: Handler,
        *,
        workers: int = 2,
        maxsize: int = 5,
        priority: bool = False,
    ):
        self._handler = handler
        self._workers = workers
        self._stop_event = threading.Event()
        self._threads: list[threading.Thread] = []

        if priority:
            from queue import PriorityQueue

            self._queue = PriorityQueue(maxsize=maxsize)
        else:
            from queue import Queue

            self._queue = Queue(maxsize=maxsize)

    def start(self) -> None:
        for i in range(self._workers):
            t = threading.Thread(
                target=self.__worker_loop,
                name=f"c{i + 1}",
            )
            t.start()
            self._threads.append(t)

    def stop(self) -> None:
        # 停产：不再接受新任务
        self._stop_event.set()

    def submit(self, task: Any) -> None:
        # 背压在这里生效：队列满会阻塞
        self._queue.put(task)

    def join(self) -> None:
        # 等待所有已提交任务处理完
        self._queue.join()
        # 等 worker 线程退出
        for t in self._threads:
            t.join()

    def __worker_loop(self):
        while True:
            # 停产 + 队列清空 → 退出
            if self._stop_event.is_set() and self._queue.empty():
                return
            try:
                task = self._queue.get(timeout=0.2)
            except queue.Empty:
                continue

            try:
                self._handler(task)
            finally:
                self._queue.task_done()


def handlerA(task: str):
    print(f"处理任务：{task}")


pool = WorkPool(handlerA, workers=2)
pool.start()
for i in ["A", "B", "C", "D", "E", "F"]:
    pool.submit(f"任务-{i}")

pool.stop()
pool.join()


处理任务：任务-A
处理任务：任务-B
处理任务：任务-C
处理任务：任务-D
处理任务：任务-E
处理任务：任务-F
